<a href="https://colab.research.google.com/github/a7482472/Major-project/blob/main/test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-fuzzy
!pip install tensorflow-model-optimization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 15.3 MB/s eta 0:00:00


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, Dense, Flatten, Dropout, Input
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import pandas as pd
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt

# Load and preprocess data
def load_data(csv_file, time_steps=10):
    data = pd.read_csv(csv_file)
    features = data[['heart_rate', 'blood_pressure', 'oxygen_saturation', 'respiratory_rate', 'temperature']].values
    labels = data['Label'].values

    # Scale features
    scaler = StandardScaler()
    features = scaler.fit_transform(features)

    X, y = [], []
    for i in range(len(features) - time_steps):
        X.append(features[i:i + time_steps])
        y.append(labels[i + time_steps - 1])  # Label for the end of each sequence

    # Reshape X for ConvLSTM2D input
    X = np.array(X).reshape(-1, time_steps, 5, 1, 1)  # time_steps, features, 1x1 spatial grid
    y = np.array(y)
    return X, y

# Load data
csv_file = '/content/CVD_Vital_SIgns.csv'  # Update with the correct path if necessary
X, y = load_data(csv_file)

# Build ConvLSTM2D model
def build_conv_lstm_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        ConvLSTM2D(filters=256, kernel_size=(1, 1), activation='relu', return_sequences=True),
        Dropout(0.2),
        ConvLSTM2D(filters=128, kernel_size=(1, 1), activation='relu', return_sequences=False),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    return model

# Define model input shape based on X
input_shape = (X.shape[1], X.shape[2], 1, 1)  # (time_steps, features, 1, 1)
model = build_conv_lstm_model(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=40, batch_size=32, validation_split=0.2)

# Define fuzzy logic system
data_variance = ctrl.Antecedent(np.arange(0, 1.1, 0.2), 'data_variance')  # Coarser granularity
anomaly_threshold = ctrl.Consequent(np.arange(0, 1.1, 0.2), 'anomaly_threshold')

# Define shifted and overlapping fuzzy membership functions
data_variance['low'] = fuzz.trimf(data_variance.universe, [0, 0.4, 0.8])  # Covers almost entire range
data_variance['medium'] = fuzz.trimf(data_variance.universe, [0.2, 0.6, 1])
data_variance['high'] = fuzz.trimf(data_variance.universe, [0.5, 0.9, 1.1])  # Slightly out of bounds

anomaly_threshold['low'] = fuzz.trimf(anomaly_threshold.universe, [0.3, 0.6, 1.1])
anomaly_threshold['medium'] = fuzz.trimf(anomaly_threshold.universe, [0, 0.5, 1.1])
anomaly_threshold['high'] = fuzz.trimf(anomaly_threshold.universe, [0, 0.3, 0.8])

# Introduce deliberately conflicting rules
rule1 = ctrl.Rule(data_variance['low'], anomaly_threshold['high'])
rule2 = ctrl.Rule(data_variance['medium'], anomaly_threshold['low'])
rule3 = ctrl.Rule(data_variance['high'], anomaly_threshold['low'])
rule4 = ctrl.Rule(data_variance['low'] & data_variance['medium'], anomaly_threshold['medium'])
rule5 = ctrl.Rule(data_variance['medium'] | data_variance['high'], anomaly_threshold['high'])

# Create control system and simulation
anomaly_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5])
anomaly_sim = ctrl.ControlSystemSimulation(anomaly_ctrl)

# Function to adjust thresholds based on historical variance
def adjust_threshold(predictions, historical_variance):
    adjusted_thresholds = []
    for variance in historical_variance:
        anomaly_sim.input['data_variance'] = variance
        anomaly_sim.compute()
        adjusted_thresholds.append(anomaly_sim.output['anomaly_threshold'])
    return np.array(adjusted_thresholds)

# After model predictions
predictions = model.predict(X)
historical_variance = np.var(X, axis=(1, 2, 3))  # Variance per sample
adjusted_thresholds = adjust_threshold(predictions.flatten(), historical_variance)
predictions_flat = predictions.flatten()
adjusted_thresholds = adjusted_thresholds.flatten()

# Ensure adjusted_thresholds matches predictions_flat length
adjusted_thresholds = adjusted_thresholds[:len(predictions_flat)]

# Convert predictions to binary labels based on degraded thresholds
binary_predictions = (predictions_flat > adjusted_thresholds).astype(int)

# Evaluate the model
def evaluate_model(y_true, y_pred, y_pred_prob):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

evaluate_model(y, binary_predictions, predictions_flat)


Epoch 1/40
587/587 ━━━━━━━━━━━━━━━━━━━━ 23s 21ms/step - accuracy: 0.7840 - loss: 0.4913 - val_accuracy: 0.9341 - val_loss: 0.1851
Epoch 2/40
587/587 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - accuracy: 0.9233 - loss: 0.2017 - val_accuracy: 0.9450 - val_loss: 0.1299
Epoch 3/40
587/587 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.9382 - loss: 0.1532 - val_accuracy: 0.9695 - val_loss: 0.0869
Epoch 4/40
587/587 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.9580 - loss: 0.1155 - val_accuracy: 0.9787 - val_loss: 0.0685
Epoch 5/40
587/587 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.9625 - loss: 0.1019 - val_accuracy: 0.9751 - val_loss: 0.0730
Epoch 6/40
587/587 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.9637 - loss: 0.0986 - val_accuracy: 0.9838 - val_loss: 0.0577
Epoch 7/40
587/587 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.9688 - loss: 0.0908 - val_accuracy: 0.9627 - val_loss: 0.0939
Epoch 8/40
587/587 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.9662 - loss: 0.0888 - val_